In [14]:
newtype Variable = V String
newtype Property = P String
newtype Role = R String

type Instance = (Variable,Property)

Here we declare a datatype for Basic Abstract Meaning Representations (Basic AMRs, Bos 2016). An AMR is either a constant, or an instance assignment with 0 or more out-going roles. Note that since the list of outgoing roles can be empty, this simplifies the syntax given by Bos.

In [15]:
data AMR = C String | I (Variable,Property) [(Role,AMR)]

Some examples of AMRs using the above datatype:

In [28]:
-- The children moaned.
-- (e / moan-01 :ARG0 (x / child))

ex1 = I (V "e", P "moan-01") [(R ":ARG0", I (V "x", P "child") [])]

-- Ms Ribble handed out envelopes to the children
-- (e / give-01 :ARG0 (x / person :named "Ms Ribble") :ARG2 (y / child) :ARG1 (z / envelope))

ex2 = I (V "e", P "give-01") [(R ":ARG0", I (V "x", P "person") [(R ":named", C "Ms. Ribble")]),(R ":ARG2", I (V "y", P "child") []),(R ":ARG1", I (V "z", P "envelope") [])]

-- It rains.
-- (e / rain-01)

ex3 = I (V "e", P "rain-01") []

We want to recursively map AMRs to First-Order Logic (FOL). Let's encode FOL in haskell.

In [75]:
data Term = Constant String | Var String

ppTerm :: Term -> String
ppTerm (Constant c) = c
ppTerm (Var v) = v

instance Show Term where
    show = ppTerm

data FoL = Atom String [Term] | Not FoL | And FoL FoL | Exists String FoL | TT | FF

instance Show FoL where
    show = ppFoL

commas = foldr1 (\w s -> w ++ ',':s)

ppFoL :: FoL -> String
ppFoL (Atom pred ts) = pred ++ "(" ++ commas (map show ts) ++ ")"
ppFoL (Not f) = "~(" ++ ppFoL f ++ ")"
ppFoL (And f g) = ppFoL f ++ " && " ++ ppFoL g
ppFoL (Exists v f) = "E" ++ v ++ "(" ++ ppFoL f ++ ")"
ppFoL TT = "T"
ppFoL FF = "F"

We want a function I that maps (i) AMR constants to FoL constants, and (ii) AMR instance assignments to FoL atomic formulas. This implements the semantics of basic AMRs from Lai et al.

In [76]:
import Control.Monad.Cont

conjoin :: [FoL] -> FoL
conjoin = foldr And TT

amrToFoL :: AMR -> Cont FoL Term
amrToFoL (C s) = return $ Constant s 

amrToFoL (I (V v,P p) rs) = cont (\k -> Exists v $
    Atom p [Var v]
    `And` k (Var v)
    `And` conjoin (map (\(R r,a) -> (runCont $ amrToFoL a) (\y -> Atom r [Var v,y])) rs))

We'll write a quick normalization function to remove redundant conjuncts:

In [77]:
normalizeFoL :: FoL -> FoL
normalizeFoL (Atom s ts) = Atom s ts
normalizeFoL (Not f) = Not (normalizeFoL f)
normalizeFoL TT = TT
normalizeFoL FF = FF
normalizeFoL (Exists v f) = Exists v (normalizeFoL f)
normalizeFoL (And f TT) = normalizeFoL f
normalizeFoL (And TT f) = normalizeFoL f
normalizeFoL (And f g) = And (normalizeFoL f) (normalizeFoL g)

We'll also need a lower function to get rid of the continuation argument. This is implicit in Lai et al's semantics.

In [78]:
lowerFoL :: Cont FoL Term -> FoL
lowerFoL k = runCont k (const TT)

Now we can test the resulting mapping function.

In [80]:
(normalizeFoL . lowerFoL . amrToFoL) ex1

Ee(moan-01(e) && Ex(child(x) && :ARG0(e,x)))

In [81]:
(normalizeFoL . lowerFoL . amrToFoL) ex2

Ee(give-01(e) && Ex(person(x) && :ARG0(e,x) && :named(x,Ms. Ribble)) && Ey(child(y) && :ARG2(e,y)) && Ez(envelope(z) && :ARG1(e,z)))

In [82]:
(normalizeFoL . lowerFoL . amrToFoL) ex3

Ee(rain-01(e))